#### Authorization and authentication

Importing libraries

In [1]:
import requests
import json
import base64
import webbrowser
import pandas as pd
from urllib.parse import urlencode
from spotify_secrets import client_id, client_secret, refresh_token

Getting the authorization code

In [ ]:
auth_headers = {
    'client_id': client_id,
    'response_type': 'code',
    'redirect_uri': 'http://localhost:7777/callback',
    'scope': 'playlist-modify-public user-follow-read playlist-read-collaborative'
    }

webbrowser.open('https://accounts.spotify.com/authorize?' + urlencode(auth_headers))

In [12]:
code = 'AQCkCXJDL9ine4d9tfJPWAbGTzzghSULpKkoctY-Op4tIcj0nvByw2J0WC_Qrv0tFiLLCFXTX0Hhth6XEAiEdBtA6mckoHRykUlKUGWhOKMrLw_Qxi-WNPNi4J4j1C0tETEPG-tMKPllEiRsZh2oG7omoMRo5GwuUryizj4rGyOU9MIXiQ4qdXTJYuphrxc6ztxAz2ISYeadnjKtE8RAsPMjQ1VddOGpbB33qTt7mINIp4U3a7mu6Z3U905jk_VNNP2_IAvjmoZ6'

Getting the access token and refresh token

In [ ]:
# Spotify requires to provide client_id and client_secret in a base64 encoded format to authorize

encoded_cred = base64.b64encode(client_id.encode() + b':' + client_secret.encode()).decode('utf-8')

token_headers = {
    'Authorization': 'Basic ' + encoded_cred,
    'Content_type': 'application/x-www-form-urlencoded'
}

token_data = {
    'grant_type': 'authorization_code',
    'code': code,
    'redirect_uri': 'http://localhost:7777/callback'
}

In [ ]:
# Get authorization token from the authorization code (first time use)

r = requests.post(
    'https://accounts.spotify.com/api/token', 
    data = token_data, 
    headers = token_headers)
token = r.json()['access_token']

In [3]:
# Get authorization token from the refresh token, not the authorization code (for the future uses)

refresh_response = requests.post(
    'https://accounts.spotify.com/api/token',
    data = {
        'grant_type':'refresh_token',
        'refresh_token':refresh_token
        },
    headers = token_headers
    )
token = refresh_response.json()['access_token']

### TOP 10 of every followed artist

Defining headers and parameters for the requests.

In [4]:
user_headers = {
    'Authorization': 'Bearer ' + token,
    'Content-Type': 'application/json'
}

user_params = {
    'limit': 50,
    'country': 'PL'
}

Due to the limitations, a list of all followed artists needs to be requested in parts of 50.

In [5]:
#get first 50 followed artists

artists_response = requests.get(
    'https://api.spotify.com/v1/me/following?type=artist', 
    params=user_params, 
    headers=user_headers)

artist_id_list = [i['id'] for i in artists_response.json()['artists']['items']]
no_of_artists = artists_response.json()['artists']['total']
next_50_artists = artists_response.json()['artists']['next']

In [6]:
#get next followed artists and make them into one list

for i in range(no_of_artists//50):
    artists_response = requests.get(
        f"{next_50_artists}", 
        params=user_params, 
        headers=user_headers)
    artist_id_list = artist_id_list + [i['id'] for i in artists_response.json()['artists']['items']]
    next_50_artists = artists_response.json()['artists']['next']

Getting the list of Top 10 songs for every followed artist. These are then combined into a long list of every artist's Top 10.

In [7]:
all_songs = []
for artist in artist_id_list:
    songs_response = requests.get(
        f'https://api.spotify.com/v1/artists/{artist}/top-tracks', 
        params=user_params, 
        headers=user_headers)
    artist_top_songs_list = [i['id'] for i in songs_response.json()['tracks']]
    all_songs = all_songs + artist_top_songs_list

Creating a new playlist that can be filled with new list of songs. Defining a function can be useful in the future with the playlist merger script.  
Function returns an ID of created playlist.

In [8]:
def create_playlist(name, description, public):
        playlist_response = requests.post(
                'https://api.spotify.com/v1/users/11171749586/playlists',
                headers = user_headers,
                json = {
                        'name': name,
                        'description': description,
                        'public': public
                        }
        )
        new_playlist_id = playlist_response.json()['id']
        return new_playlist_id

new_playlist_id = create_playlist('Followed Top 10', "All followed artists' Top 10", True)

Adding songs to created playlist. Also with defined function and using the ID obtained before.

In [9]:
def add_songs_to_playlist (songs_list, playlist_id):

        #add required strings to the songs IDs (songs IDs need to be in a specific format)
        all_songs_stringed = ['spotify:track:' + song for song in songs_list]

        #create strings of songs IDs in groups of 100 (maximum number that can be added with one request)
        songs_pack = [all_songs_stringed[x:x+100] for x in range(0, len(all_songs_stringed), 100)]

        for strings in songs_pack:
                add_items_response = requests.post(
                        f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks',
                        headers = user_headers,
                        json = {
                                'uris': strings
                        }
                );

add_songs_to_playlist(all_songs, new_playlist_id)

### Playlists Merger

Getting a list of all the playlists that were created by a user.  
Using Pandas to create a table for comfortable view and easy ID's picking.

In [10]:
playlists_response = requests.get(
    'https://api.spotify.com/v1/users/11171749586/playlists', 
    params=user_params, 
    headers=user_headers)

playlists_list = playlists_response.json()

In [13]:
playlists_df = pd.DataFrame({
                            'name':[i['name'] for i in playlists_list['items']],
                            'id':[i['id'] for i in playlists_list['items']]
})

Picking playlists for merging into new Car Playlist

In [14]:
merge_list = ['47PYFKmPotdMtXDOLm2o7s', '5QBr683bl4i9dwMQcFwDvT', '11QN76qqTLW9VY2qjicuNP', '34109C2GcEAmTp5biAKnoM', '51rebyQvvjW7w8S1vHvIyz', '51NO4S2XoJM88QacOMblKC']

Getting a list of songs from a list of playlists. Defining a function so it can be used with to merge different playlists in the future.

In [15]:
def get_playlists_songs(playlists_id_list):
    
    songs_id_list = []

    for id in playlists_id_list:
        id_response = requests.get(
                            f'https://api.spotify.com/v1/playlists/{id}/tracks', 
                            params=user_params, 
                            headers=user_headers)
    
        temp_songs_id_list = [i['track']['id'] for i in id_response.json()['items']]
        no_of_songs = id_response.json()['total']
        next_50_songs = id_response.json()['next']

        if no_of_songs > 50:
            for i in range(no_of_songs//50):
                id_response = requests.get(
                    f"{next_50_songs}", 
                    params=user_params, 
                    headers=user_headers)
                temp_songs_id_list = temp_songs_id_list + [i['track']['id'] for i in id_response.json()['items']]
                next_50_songs = id_response.json()['next']

        songs_id_list = songs_id_list + temp_songs_id_list

    return songs_id_list

In [16]:
all_songs_list = get_playlists_songs(merge_list)

Creating a new playlist and filling it with the list of songs. Using functions that were defined earlier.

In [17]:
car_playlist_id = create_playlist('Car playlist', 'Songs for driving', True)

In [18]:
add_songs_to_playlist(all_songs_list, car_playlist_id)